In [ ]:
import pathlib

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys

In [ ]:
root_monaco = pathlib.Path(r'W:\SASH\1~Clinical\1~196309\plan')
tel_files = list(root_monaco.glob('*/tel.1'))
tel_files

In [ ]:
root_logfiles = pathlib.Path(r'D:\LinacLogFiles\indexed')
patient_id = '999993'
field_label = '9309A'

logfile_paths_a = list(root_logfiles.glob(f'*/{patient_id}_*/*/*{field_label}*/*/*.trf'))
logfile_paths_a

In [ ]:
field_label = '9309B'

logfile_paths_b = list(root_logfiles.glob(f'*/{patient_id}_*/*/*{field_label}*/*/*.trf'))
logfile_paths_b

In [ ]:
all_logfiles = logfile_paths_a + logfile_paths_b

deliveries = {}
for path in all_logfiles:
    timestamp = path.parent.name
    beam_name = path.parent.parent.name
    
    key = (beam_name, timestamp)
    deliveries[key] = pymedphys.Delivery.from_logfile(path)

In [ ]:
deliveries.keys()

In [ ]:
mudensities = {}

for key, delivery in deliveries.items():
    mudensities[key] = delivery.mudensity()

In [ ]:
monaco_deliveries = [
    pymedphys.Delivery.from_monaco(tel_file) for tel_file in tel_files
]


In [ ]:
monaco_mudensities = [
    delivery.mudensity() for delivery in monaco_deliveries
]

In [ ]:
GRID = pymedphys.mudensity.grid()
COORDS = (GRID["jaw"], GRID["mlc"])
COLOUR_MAGNITUDE = 10

In [ ]:
def run_comparison(evaluation, reference):
    diff = evaluation - reference
    print(
        f"Mean:   {np.mean(diff)}\n"
        f"Median: {np.median(diff)}\n"
        f"Std:    {np.std(diff)}\n"
        f"Max:    {np.max(diff)}\n"
        f"Min:    {np.min(diff)}\n"
    )
    pymedphys.mudensity.display(
        GRID, diff, cmap="seismic", vmin=-COLOUR_MAGNITUDE, vmax=COLOUR_MAGNITUDE
    )
    
    return diff

In [ ]:
monaco_map = {
    '86951_9309A_CW_VMAT': 0,
    '86961_9309B_CW-Replan_VMAT': 1
}

In [ ]:
tuning_type = {
    '2020-01-08_144045_2619': 'tuned',
    '2020-01-08_145240_2619': 'tuned',
    '2020-01-08_151409_2619': 'tuned',
    '2020-01-10_172546_2619': 'original',
    '2020-01-10_152553_2619': 'tuned',
    '2020-01-10_164453_2619': 'original'
}

In [ ]:
diffs = []
plan_names = []
timestamps = []
keys = list(deliveries.keys())[0:-1]
all_tuning = []

for key in keys:
    plan_name, timestamp = key
    monaco_mudensity = monaco_mudensities[monaco_map[plan_name]]
    logfile_mudensity = mudensities[key]
    
    diff = logfile_mudensity - monaco_mudensity
    diff = diff[monaco_mudensity > 20].tolist()
    
    tuning = tuning_type[timestamp]
    
    diffs += diff
    plan_names += [plan_name,]*len(diff)
    timestamps += [timestamp,]*len(diff)
    all_tuning += [tuning,]*len(diff)


In [ ]:
diffs = np.array(diffs).astype(float)
plan_names = np.array(plan_names).astype(str)
all_tuning = np.array(all_tuning).astype(str)

df = pd.DataFrame(
    data={
        'MU Density [Logfile - Monaco]': diffs,
        'Plan Name': plan_names,
        'Tuning': all_tuning
    }
)
df

In [ ]:
plt.figure(figsize=(10,10))
sns.violinplot(
    x='Plan Name', y='MU Density [Logfile - Monaco]', hue="Tuning", 
    data=df, palette="muted", split=True, inner='quartile')

In [ ]:
plt.figure(figsize=(10,10))
sns.violinplot(
    x='Plan Name', y='MU Density [Logfile - Monaco]', hue="Tuning", 
    data=df, palette="muted", split=True, inner='quartile')

plt.ylim([-2, 2])